In [1]:
import os
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F

In [2]:
datadir = "/home/karthiktiwari/Downloads/Beer Data"

In [3]:
beer_train = pd.read_csv(os.path.join(datadir,"beer_profile_and_ratings.csv"), index_col=0)

In [4]:
beer_train.head(5)

,Style,Brewery,Beer Name (Full),Description,ABV,Min IBU,Max IBU,Astringency,Body,Alcohol,...,Fruits,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,review_overall,number_of_reviews
Name,,,,,,,,,,,,,,,,,,,,,
Amber,Altbier,Alaskan Brewing Co.,Alaskan Brewing Co. Alaskan Amber,"Notes:Richly malty and long on the palate, wit...",5.3,25,50,13,32,9,...,33,57,8,111,3.498994,3.636821,3.556338,3.643863,3.847082,497
Double Bag,Altbier,Long Trail Brewing Co.,Long Trail Brewing Co. Double Bag,"Notes:This malty, full-bodied double alt is al...",7.2,25,50,12,57,18,...,24,35,12,84,3.798337,3.846154,3.904366,4.024948,4.034304,481
Long Trail Ale,Altbier,Long Trail Brewing Co.,Long Trail Brewing Co. Long Trail Ale,Notes:Long Trail Ale is a full-bodied amber al...,5.0,25,50,14,37,6,...,10,54,4,62,3.409814,3.667109,3.600796,3.631300,3.830239,377
Doppelsticke,Altbier,Uerige Obergärige Hausbrauerei GmbH / Zum Uerige,Uerige Obergärige Hausbrauerei GmbH / Zum Ueri...,Notes:,8.5,25,50,13,55,31,...,49,40,16,119,4.148098,4.033967,4.150815,4.205163,4.005435,368
Sleigh'r Dark Doüble Alt Ale,Altbier,Ninkasi Brewing Company,Ninkasi Brewing Company Sleigh'r Dark Doüble A...,Notes:Called 'Dark Double Alt' on the label.Se...,7.2,25,50,25,51,26,...,11,51,20,95,3.625000,3.973958,3.734375,3.765625,3.817708,96


In [5]:
beer_train.columns

Index(['Style', 'Brewery', 'Beer Name (Full)', 'Description', 'ABV', 'Min IBU',
       'Max IBU', 'Astringency', 'Body', 'Alcohol', 'Bitter', 'Sweet', 'Sour',
       'Salty', 'Fruits', 'Hoppy', 'Spices', 'Malty', 'review_aroma',
       'review_appearance', 'review_palate', 'review_taste', 'review_overall',
       'number_of_reviews'],
      dtype='object')

In [6]:
beer_train[beer_train['ABV']>57]

,Style,Brewery,Beer Name (Full),Description,ABV,Min IBU,Max IBU,Astringency,Body,Alcohol,...,Fruits,Hoppy,Spices,Malty,review_aroma,review_appearance,review_palate,review_taste,review_overall,number_of_reviews
Name,,,,,,,,,,,,,,,,,,,,,
Schorschbock 57%,Bock - Eisbock,Schorschbräu,Schorschbräu Schorschbock 57%,Notes:,57.5,25,35,0,7,17,...,10,0,2,4,4.0,4.0,4.0,3.5,4.0,1


In [ ]:
class BeerDataset(Dataset):
    def __init__(self, df, config):
        """
        Custom dataset class for reading beer attributes and review score from dataframe

        Args:
            df (Pandas DataFrame): Pandas Dataframe with Beer Data
        """
        self.df = df
        self.config = config

    def __getitem__(self, index):
        for attr in self.config['attrs']:
            pass
        
        return None


    def __len__(self):
        return len(self.df)